<a href="https://colab.research.google.com/github/AdamMcCloskey381/machine-learning/blob/main/Perceptron_pipeline_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler 
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
iris = load_iris()

df = pd.DataFrame(data = np.c_[iris["data"], iris["target"]],
                  columns = iris.feature_names + ["target"])
df.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


In [3]:
x = df.loc[:, df.columns != "target"]
y = df.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 106)

In [4]:
num_col = df.columns[0:len(df.columns)-1].tolist()
num_trans = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median")), ("Scaler", StandardScaler())])
preproc = ColumnTransformer(transformers = [("num_trans", num_trans, num_col)])

In [5]:
pipeline = Pipeline(steps = [("preprocessing", preproc), ("model", Perceptron())])

from sklearn import set_config
set_config(display = "diagram")
pipeline

pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_trans',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('Scaler',
                                                                   StandardScaler())]),
                                                  ['sepal length (cm)',
                                                   'sepal width (cm)',
                                                   'petal length (cm)',
                                                   'petal width (cm)'])])),
                ('model', Perceptron())])

In [6]:
score = accuracy_score(y_test, pipeline.predict(x_test))
round(score, 2)

0.95

In [7]:
with open("model.pkl", "wb") as fh:
  pickle.dump(pipeline, fh)

In [8]:
pnp = Perceptron()
pnp.fit(x_train, y_train)
with open("pnp.pkl", "wb") as fh:
  pickle.dump(pnp, fh)

In [9]:
%%file pnp_predict.py

from flask import Flask, request
import pickle
import numpy as np
from sklearn.linear_model import Perceptron

app = Flask(__name__)

@app.route("/pnp_pred", methods = ["GET"])

def get_pred():
  sl = request.args.get("sl", 0, type = float)
  sw = request.args.get("sw", 0, type = float)
  pl = request.args.get("pl", 0, type = float)
  pw = request.args.get("pw", 0, type = float)

  data = np.array([sl, sw, pl, pw]).reshape(1, -1)
  
  with open("pnp.pkl", "rb") as fh:
    model = pickle.load(fh)
  
  prediction = model.predict(data)[0]

  return {"prediction": prediction, "features": {"sl": sl, "sw": sw, "pl": pl, "pw": pw}}

if __name__ == "__main__":
  app.run()

Writing pnp_predict.py


In [10]:
import subprocess
p = subprocess.Popen(["python", "pnp_predict.py"])

In [13]:
import requests
response = requests.get("http://127.0.0.1:5000/pnp_pred")
response.json()

{'features': {'pl': 0, 'pw': 0, 'sl': 0, 'sw': 0}, 'prediction': 1.0}

In [14]:
p.kill()